In [137]:
from nltk import FreqDist
from gensim import corpora, models
from gensim.utils import simple_preprocess

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
%matplotlib inline

pyLDAvis.enable_notebook()

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reference: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [3]:
gcash_good = pd.read_csv('/Mynt Capstone/repo/capstone-project/Data/Preprocessed/Mico/App Store/gcash_good.csv')
gcash_bad = pd.read_csv('/Mynt Capstone/repo/capstone-project/Data/Preprocessed/Mico/App Store/gcash_bad.csv')
# , index_col=[0]

In [4]:
gcash_good.head()

,date,rating,title_review
0,2020-01-10 03:29:09,5,"['nice', 'far', 'good', 'convenient', 'problem..."
1,2020-01-17 17:59:30,5,"['good', 'service', 'plz', 'support']"
2,2020-01-20 17:32:16,4,"['incoming', 'outgo', 'transaction', 'limit', ..."
3,2020-01-23 00:33:47,5,"['convenience', 'since', 'day', 'instal', 'app..."
4,2020-01-25 06:41:20,4,"['icon', 'ok', 'maybe', 'stop', 'rearrange', '..."


In [7]:
gcash_good.dtypes

date            object
rating           int64
title_review    object
dtype: object

In [6]:
gcash_bad.head()

,date,rating,title_review
0,2020-01-01 02:10:56,3,"['need', 'improvement', 'great', 'try', 'buy',..."
1,2020-01-01 11:26:33,1,"['total', 'scam', 'andami', 'video', 'youtube'..."
2,2020-01-01 15:11:30,3,"['email', 'registration', 'error', 'log', 'ema..."
3,2020-01-01 19:14:19,1,"['customer', 'support', 'issue', 'trash', 'avi..."
4,2020-01-02 01:43:34,1,"['outdate', 'please', 'keep', 'bad', 'dial']"


In [87]:
gcash_rating5 = gcash_good[gcash_good['rating'] == 5]
gcash_rating5.head()

,date,rating,title_review
0,2020-01-10 03:29:09,5,"['nice', 'far', 'good', 'convenient', 'problem..."
1,2020-01-17 17:59:30,5,"['good', 'service', 'plz', 'support']"
3,2020-01-23 00:33:47,5,"['convenience', 'since', 'day', 'instal', 'app..."
6,2020-02-07 18:34:41,5,"['face', 'id', 'authenticator', 'hope', 'could..."
7,2020-02-10 02:45:27,5,"['grest', 'improvement', 'year', 'use', 'sever..."


In [11]:
gcash_rating1 = gcash_bad[gcash_bad['rating'] == 1]
gcash_rating1.head()

,date,rating,title_review
1,2020-01-01 11:26:33,1,"['total', 'scam', 'andami', 'video', 'youtube'..."
3,2020-01-01 19:14:19,1,"['customer', 'support', 'issue', 'trash', 'avi..."
4,2020-01-02 01:43:34,1,"['outdate', 'please', 'keep', 'bad', 'dial']"
5,2020-01-02 05:52:51,1,"['refuse', 'let', 'cash', 'try', 'many', 'time..."
6,2020-01-02 08:15:18,1,"['worthless', 'download', 'register', 'last', ..."


In [88]:
def to_sentence(str_list):
    new_str_list = str_list.replace("'", "")
    words = new_str_list.strip('][').split(', ')
    return words

In [111]:
gcash_rating5['title_review'] = gcash_rating5['title_review'].apply(lambda x: to_sentence(x))
gcash_rating1['title_review'] = gcash_rating1['title_review'].apply(lambda x: to_sentence(x))

<ipython-input-111-0a361396980a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gcash_rating1['title_review'] = gcash_rating1['title_review'].apply(lambda x: to_sentence(x))


In [90]:
gcash_rating5.head()

,date,rating,title_review
0,2020-01-10 03:29:09,5,"[nice, far, good, convenient, problem, encount..."
1,2020-01-17 17:59:30,5,"[good, service, plz, support]"
3,2020-01-23 00:33:47,5,"[convenience, since, day, instal, apps, always..."
6,2020-02-07 18:34:41,5,"[face, id, authenticator, hope, could, add, fa..."
7,2020-02-10 02:45:27,5,"[grest, improvement, year, use, several, year,..."


In [112]:
gcash_rating1.head()

,date,rating,title_review
1,2020-01-01 11:26:33,1,"[total, scam, andami, video, youtube, right, s..."
3,2020-01-01 19:14:19,1,"[customer, support, issue, trash, avid, user, ..."
4,2020-01-02 01:43:34,1,"[outdate, please, keep, bad, dial]"
5,2020-01-02 05:52:51,1,"[refuse, let, cash, try, many, time, cash, eve..."
6,2020-01-02 08:15:18,1,"[worthless, download, register, last, monday, ..."


### Creating the LDA Model <br>
### 5-Star Reviews

In [126]:
dictionary = corpora.Dictionary(gcash_rating5['title_review'])

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [127]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [128]:
bow_corpus = [dictionary.doc2bow(doc) for doc in gcash_rating5['title_review']]

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [129]:
bow_corpus[2]

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]

In [130]:
bow_doc_2 = bow_corpus[2]
for i in range(len(bow_doc_2)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                               dictionary[bow_doc_2[i][0]], 
bow_doc_2[i][1]))

Word 5 ("payment") appears 1 time.
Word 6 ("bill") appears 1 time.
Word 7 ("help") appears 1 time.
Word 8 ("load") appears 1 time.
Word 9 ("money") appears 1 time.
Word 10 ("time") appears 1 time.
Word 11 ("transfer") appears 1 time.
Word 12 ("use") appears 1 time.


D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [131]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=7, id2word=dictionary, passes=2, workers=2)

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [132]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.180*"pay" + 0.108*"bill" + 0.072*"make" + 0.067*"transaction" + 0.058*"convenient" + 0.054*"like" + 0.053*"use" + 0.047*"go" + 0.044*"transfer" + 0.036*"easy"
Topic: 1 
Words: 0.134*"cash" + 0.083*"transaction" + 0.079*"payment" + 0.079*"use" + 0.073*"convenient" + 0.060*"thank" + 0.053*"bank" + 0.050*"money" + 0.047*"get" + 0.045*"time"
Topic: 2 
Words: 0.199*"use" + 0.062*"easy" + 0.061*"bank" + 0.056*"transfer" + 0.056*"convenient" + 0.055*"transaction" + 0.049*"load" + 0.040*"online" + 0.037*"I" + 0.036*"money"
Topic: 3 
Words: 0.153*"money" + 0.099*"love" + 0.095*"I" + 0.058*"useful" + 0.054*"pay" + 0.050*"great" + 0.047*"bank" + 0.047*"transaction" + 0.043*"get" + 0.041*"make"
Topic: 4 
Words: 0.197*"good" + 0.157*"update" + 0.079*"please" + 0.050*"thank" + 0.048*"time" + 0.048*"use" + 0.046*"payment" + 0.043*"make" + 0.034*"need" + 0.032*"like"
Topic: 5 
Words: 0.176*"need" + 0.122*"help" + 0.099*"thank" + 0.082*"money" + 0.082*"good" + 0.075*"great" + 0.051*"

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [134]:
lda_viz = gensimvis.prepare(lda_model, bow_corpus, dictionary)

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [138]:
lda_viz

D:\Anaconda3\envs\project-env-2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.074542 -0.013618       1        1  23.013887
0     -0.123121 -0.006737       2        1  21.251056
3     -0.055904 -0.094155       3        1  16.316587
4      0.016580  0.169901       4        1  14.043648
1     -0.073378  0.009752       5        1  10.704294
5      0.167565  0.003743       6        1   9.150274
6      0.142800 -0.068886       7        1   5.520254, topic_info=          Term       Freq      Total Category  logprob  loglift
14        love  33.000000  33.000000  Default  29.0000  29.0000
2         hope  19.000000  19.000000  Default  28.0000  28.0000
1         good  40.000000  40.000000  Default  27.0000  27.0000
13        need  23.000000  23.000000  Default  26.0000  26.0000
23        cash  25.000000  25.000000  Default  25.0000  25.0000
..         ...        ...        ...      ...      ...      ...
18        bank   0.161955  30.854417   Topic7  -5.6511  -2.3530
6         bill   0.162318  35.813776   Topic7  -5.6489  -2.4998
0   convenient   0.162513  36.515394   Topic7  -5.6477  -2.5180
1         good   0.164171  40.258403   Topic7  -5.6375  -2.6054
16         pay   0.162637  49.101974   Topic7  -5.6469  -2.8134

[232 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
15        1  0.251063       I
15        2  0.143465       I
15        3  0.466260       I
15        4  0.107599       I
15        5  0.035866       I
...     ...       ...     ...
12        7  0.015393     use
25        1  0.123849  useful
25        2  0.185774  useful
25        3  0.495397  useful
25        4  0.185774  useful

[169 rows x 3 columns], R=29, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 5, 2, 6, 7])

### Using TFIDF

In [109]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [110]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.146*"money" + 0.109*"hope" + 0.092*"transaction" + 0.084*"update" + 0.079*"need" + 0.071*"thank" + 0.062*"help" + 0.046*"use" + 0.044*"like" + 0.037*"transfer"
Topic: 1 Word: 0.171*"time" + 0.079*"load" + 0.077*"thank" + 0.066*"great" + 0.066*"love" + 0.045*"use" + 0.039*"pay" + 0.039*"make" + 0.038*"convenient" + 0.030*"need"
Topic: 2 Word: 0.117*"pay" + 0.101*"bill" + 0.099*"please" + 0.087*"help" + 0.070*"convenient" + 0.054*"great" + 0.042*"online" + 0.034*"update" + 0.033*"go" + 0.033*"transfer"
Topic: 3 Word: 0.089*"cash" + 0.081*"use" + 0.079*"useful" + 0.066*"payment" + 0.058*"bank" + 0.055*"get" + 0.052*"like" + 0.045*"I" + 0.037*"thank" + 0.037*"pay"
Topic: 4 Word: 0.150*"good" + 0.070*"love" + 0.068*"use" + 0.063*"make" + 0.058*"transaction" + 0.054*"I" + 0.050*"easy" + 0.049*"convenient" + 0.043*"like" + 0.042*"money"


### Add to stopwords: "I", "thank", "please"

### 1-Star Reviews

In [113]:
dictionary = corpora.Dictionary(gcash_rating1['title_review'])

In [114]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [115]:
bow_corpus = [dictionary.doc2bow(doc) for doc in gcash_rating1['title_review']]

In [118]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=2, workers=2)

In [119]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.030*"use" + 0.026*"update" + 0.020*"try" + 0.019*"money" + 0.019*"load" + 0.017*"pay" + 0.015*"I" + 0.015*"time" + 0.014*"even" + 0.014*"bill"
Topic: 1 
Words: 0.041*"id" + 0.021*"cash" + 0.020*"nyo" + 0.020*"yung" + 0.018*"student" + 0.017*"verification" + 0.016*"naman" + 0.014*"service" + 0.014*"load" + 0.013*"please"
Topic: 2 
Words: 0.031*"money" + 0.030*"service" + 0.028*"account" + 0.027*"use" + 0.025*"customer" + 0.018*"transaction" + 0.016*"work" + 0.014*"email" + 0.013*"poor" + 0.013*"cash"
Topic: 3 
Words: 0.042*"verify" + 0.040*"account" + 0.032*"fix" + 0.030*"update" + 0.027*"use" + 0.022*"get" + 0.021*"please" + 0.021*"try" + 0.015*"money" + 0.015*"need"


### Using TFIDF

In [121]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [122]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.017*"cash" + 0.016*"paypal" + 0.015*"service" + 0.014*"update" + 0.012*"verification" + 0.012*"always" + 0.011*"id" + 0.011*"mpin" + 0.011*"unavailable" + 0.010*"customer"
Topic: 1 Word: 0.023*"id" + 0.021*"verify" + 0.018*"account" + 0.015*"work" + 0.013*"fully" + 0.012*"fix" + 0.012*"use" + 0.012*"log" + 0.011*"please" + 0.011*"try"
Topic: 2 Word: 0.015*"money" + 0.014*"load" + 0.013*"pera" + 0.013*"transaction" + 0.013*"scam" + 0.012*"code" + 0.011*"update" + 0.011*"gcredit" + 0.011*"receive" + 0.011*"nyo"
Topic: 3 Word: 0.017*"verify" + 0.017*"use" + 0.016*"update" + 0.012*"account" + 0.011*"get" + 0.011*"load" + 0.011*"money" + 0.010*"pay" + 0.010*"time" + 0.009*"need"


In [123]:
!pip install pyLDAvis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=d0a19943401247c6b03fa4939e6094d1605cd20aac3d71c415cf7688f841242d
  Stored in directory: c:\users\nicolas da silva\appdata\local\pip\cache\wheels\57\a4\86\d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=09f4d756530242115530e8092ea0fcb61b1978863ab7ca7e827104ada07ab40d
  Stored in directory: c:\users\nicolas da silva\appdata\local\pip\cache\wheels\2f\a0\d3\4030d9f80e6b3be787f19fc911b8e7aa462986a40a

In [ ]:
lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)